In [5]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from IPython.display import clear_output

from src.torch.gridgen import gridgen
from src.torch.gridgen import mygriddata
from src.torch.gridgen import fast_sine_transform_y
import torch
import matplotlib.pyplot as plt

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
def display_im_grid(xgrid, ygrid, im, ngrid):
    """Display both grids and image"""
    clear_output(wait=True)
    plt.imshow(im, cmap="gray")
    plt.plot(ygrid[::ngrid, ::ngrid], xgrid[::ngrid, ::ngrid], "b", lw=1.0)
    plt.plot(ygrid[::ngrid, ::ngrid].T, xgrid[::ngrid, ::ngrid].T, "b", lw=1.0)

    plt.axis("off")
    plt.axis("equal")
    plt.pause(0.1)

In [10]:
"""Example code"""
im = torch.zeros((10, 2, 81, 121), device=device)

bsz, c, szx, szy = im.shape

im[:, :, ::5, :], im[:, :, :, ::5] = 1, 1
im[:, :, ::5, ::5] = 1

nframes = bsz
j_lb = 0.3
j_ub = 3.0

f11 = torch.linspace(1.0, 0.3, nframes, device=device)
f12 = torch.linspace(1.0, 3.0, nframes, device=device)
f21 = torch.linspace(0.0, 1.0, nframes, device=device)
f22 = torch.linspace(0.0, -1.0, nframes, device=device)

cz = int(0.1 * min(szx, szy))

f = torch.ones((bsz, 2, szx, szy), device=device, requires_grad=True)
f[:, 1, :, :] = 0

for i in range(nframes):
    f[
        i, 0, szx // 3 - 1 - cz : szx // 3 + cz, szy // 3 - 1 - cz : szy // 3 + cz
    ] = f11[i]
    f[
        i,
        0,
        2 * szx // 3 - 1 - cz : 2 * szx // 3 + cz,
        2 * szy // 3 - 1 - cz : 2 * szy // 3 + cz,
    ] = f12[i]
    f[
        i, 1, szx // 3 - 1 - cz : szx // 3 + cz, szy // 3 - 1 - cz : szy // 3 + cz
    ] = f21[i]
    f[
        i,
        1,
        2 * szx // 3 - 1 - cz : 2 * szx // 3 + cz,
        2 * szy // 3 - 1 - cz : 2 * szy // 3 + cz,
    ] = f22[i]

pos = gridgen(f, j_lb, j_ub, inv=False)
pos_inv = gridgen(f, j_lb, j_ub, inv=True)

im[:, 0:1, :, :] = mygriddata(pos_inv, im[:, 0:1, :, :])

np_pos = pos.detach().cpu().numpy()
np_imw = im.detach().cpu().numpy()

print ('np_pos: ' + str(np_pos.shape))

for i in range(nframes):
    print ('frame: ' + str(i))
    np_pos_frame = np_pos[i,:,:]
    np_min = np.min(np_pos_frame)
    np_max = np.max(np_pos_frame)
    print ('min: ' + str(np_min) + ' max: ' + str(np_max))
    

# for i in range(nframes):
#     display_im_grid(
#         (szx - 1) * (np_pos[i, 0] + 1) / 2,
#         (szy - 1) * (np_pos[i, 1] + 1) / 2,
#         np_imw[i, 0],
#         5,
#     )


np_pos: (10, 2, 81, 121)
frame: 0
min: -1.0 max: 1.0
frame: 1
min: -1.0001192 max: 1.000736
frame: 2
min: -1.0002381 max: 1.001462
frame: 3
min: -1.0003551 max: 1.0021766
frame: 4
min: -1.0004709 max: 1.0028808
frame: 5
min: -1.0005856 max: 1.0035732
frame: 6
min: -1.000699 max: 1.004254
frame: 7
min: -1.0008111 max: 1.0049226
frame: 8
min: -1.000922 max: 1.0055795
frame: 9
min: -1.0010316 max: 1.0062233
